This experiment is done on the ground truth. The ground truth is created based on the NMVW constituents' display names and the rdfs:labels retrieved from the wikidata. It only contains constituesnts where the wikidata identifies is known.

In [1]:
import sys
sys.path.append("..") 

In [16]:
import pickle, pandas
from data_matching import exact_string_match, match_lastName, match_with_abbreviation
from calculate_result import calculate_result

In [3]:
with open('data/ground_truth.pkl', 'rb') as file:
    data = pickle.load(file)

In [4]:
candidates = set([str(item) for _, row in data.iterrows() for item in row['wiki_label']])

In [5]:
print(f"The size of ground truth: {len(data.index)} and size of candidates: {len(list(set(candidates)))}")

The size of ground truth: 6178 and size of candidates: 11501


# Exact string matching

In [ ]:
# Exact string matching
result_exact = exact_string_match(data, candidates)
exact_match = len(result_exact.loc[result_exact["match"] == "YES"])
print(f"The number of positive match is {exact_match}")

In [ ]:
with open("results/NMVWtoWikiExactMatchResults.pkl", "wb") as handle:
    pickle.dump(result_exact, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df = pandas.read_pickle("results/NMVWtoWikiExactMatchResults.pkl")
print(len(df.index))

In [ ]:
from calculate_result import calculate_result
calculate_result("results/NMVWtoWikiExactMatchResults.pkl")

# Surname Matching

In [6]:
result_surname = match_lastName(data, candidates)
surname_match = len(result_surname.loc[result_surname["match"] == "YES"])
print(f"The number of positive match is {surname_match}")

The number of positive match is 5245


In [7]:
with open("results/NMVWtoWikiSurnameMatchResults.pkl", "wb") as handle:
    pickle.dump(result_surname, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
df = pandas.read_pickle("results/NMVWtoWikiSurnameMatchResults.pkl")
print(len(df.index))

6178


In [9]:
from calculate_result import calculate_result
calculate_result("results/NMVWtoWikiSurnameMatchResults.pkl")

Total query: 6178 
Correct correspondence count: 4991
Recall: 0.8078666235027517


Total query: 6178
Total retrieved: 9613
Correct correspondence count: 5833 
Precision: 0.6067824820555497


F-measure: 0.6930330822714871



(6178, 9613, 5833, 0.8078666235027517, 0.6067824820555497, 0.6930330822714871)

# Abbreviation Matching

In [10]:
def match_with_abbreviation(df, candidate_list):
    def get_initials(name):
        if len(name.split(" ")) < 1:
            print(name)
            return

        names = name.split(' ')
        try:
            return ' '.join([f"{n[0]}." for n in names])
        except Exception as e:
            print(f'{name} has {str(e)} error')

    abbreviation_list = list()
    for i, row in df.iterrows():
        firstnames = get_initials(" ".join(str(row['name_label']).split(" ")[:-1]))
        if firstnames:
            initial_and_surname = firstnames + " " + str(row['name_label']).split(" ")[-1]
        else:
            initial_and_surname = str(row['name_label']).split(" ")[-1]
        # print(f"{str(row['name_label'])} --> {initial_and_surname}")
        abbreviation_list.append(initial_and_surname)

    temp_df = pandas.DataFrame({'abbreviations': abbreviation_list})
    df = pandas.concat([df, temp_df], axis=1)
    
    abbrv_candidate_list = list()
    for item in candidate_list:
        firstnames = get_initials(" ".join(str(item).split(" ")[:-1]))
        if firstnames:
            initial_and_surname = firstnames + " " + str(row).split(" ")[-1]
        else:
            initial_and_surname = str(row).split(" ")[-1]
        abbrv_candidate_list.append(initial_and_surname)
    
    match = ["NO" for i in range(len(df.index))]
    match_results = []
    # matching
    for i, row in df.iterrows():
        row_match_results = list()
        row_match_results = list(set([str(row['abbreviations'])]).intersection(set(abbrv_candidate_list)))
        if len(row_match_results) > 0:
            match[i] = "YES"

        match_results.append(row_match_results)
        
    temp_df = pandas.DataFrame({'retrieved_names': match_results, 'match':match})
    result_table = pandas.concat([df, temp_df], axis=1)

    return result_table

In [ ]:
import pickle, pandas
from data_matching import exact_string_match, match_lastName, match_with_abbreviation
from calculate_result import calculate_result

In [19]:
abbr = match_with_abbreviation(data[:10], candidates)

In [20]:
len(result.loc[result["match"] == "YES"])

AttributeError: 'NoneType' object has no attribute 'loc'

In [ ]:
with open("results/NMVWtoWikiSurnameMatchResults.pkl", "wb") as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df = pandas.read_pickle("results/NMVWtoWikiSurnameMatchResults.pkl")
print(len(df.index))

In [ ]:
from calculate_result import calculate_result
calculate_result("results/NMVWtoWikiSurnameMatchResults.pkl")